In [112]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
import re
from datetime import timedelta
from collections import Counter

# Load data from MongoDB

In [113]:
from pymongo import MongoClient
db_config= {
"MONGODB_ADDRESS"    : os.environ.get('KEY_MONGO_ADDRESS','mongodb-service.stag-data.svc.cluster.local'),
"MONGODB_PORT"       : os.environ.get('KEY_MONGO_PORT', '27017'),
"MONGODB_USERNAME"   : os.environ.get('KEY_MONGO_USER', 'IBSO-read-write'),
"MONGODB_PASSWORD"   : os.environ.get('KEY_MONGO_PASSWORD', 'NHWD5bw0xcMgy1Y'),
"MONGODB_DATABASE"   : os.environ.get('KEY_MONGO_DATABASE', 'admin')
}
mongo_client = MongoClient(
                ('mongodb://%s:%s@' + db_config["MONGODB_ADDRESS"] + ':' + db_config["MONGODB_PORT"] + '/' + db_config["MONGODB_DATABASE"]) % (
                    db_config["MONGODB_USERNAME"], db_config["MONGODB_PASSWORD"]), replicaset='rs0')
infodocs_metadata = mongo_client["IBSO"]["infodocs_metadata"]
metadata_collection = mongo_client["IBSO"]["infodocs_metadata"]
coll_i = mongo_client["main"]["infodocs_parsed_version"]

# ISM (Intelligent Solution Matching) API 

In [147]:
# set input values
incidentID = '002075129500003089632022'
solutionID = "3188945"


In [151]:
def get_incident_solutions_2(incidentID):
    # get details of incidentID from MongoDB
    css_object_id = incidentID
    query = {
        'css_object_id': css_object_id
    }

    res_list = list(coll_i.find(query).sort('incident_created_at_dt', -1).limit(1))

    # build payload in the right format for the ISM API
    payload = {}
    names = {'css_object_id', 'component', 'description', 'problem_description'}
    for x in res_list:
        payload = { key:value for key,value in x.items() if key in names}
    payload['queryId'] = payload.pop('css_object_id')
    payload['problemDescription'] = payload.pop('description')
    payload['stepsToReproduce'] = payload.pop('problem_description')
    payload["firstEntry"] =  0
    payload["lastEntry"] = 10

    res = requests.post('http://ism-api-service-dev.ism-api.svc.cluster.local/',json=payload)
    res_json = res.json()
    df_res = pd.json_normalize(res_json, record_path = ['result_list'])
    solution_list = df_res['nr']
    return solution_list

In [152]:
get_incident_solutions_2(incidentID)

0    2684000
1    2863760
2    2847365
3    2958935
4    3004412
5    2856023
6    3139614
7    2769192
8    2779873
9    2846783
Name: nr, dtype: object

In [54]:
incidentID = "002075129500003089632022"
get_incident_solutions(incidentID)

0    3188945
1    3191869
2    3126234
3    3082432
4    3189594
5    3016569
6    3007764
7    3027560
8    2994291
9    3007076
Name: nr, dtype: object

# Get a list of Incident IDs for a doublet 


In [153]:
customer_no = '0002348832'
installation_no = '0090463694'

In [154]:
def get_doublet_incidents(customer_no, installation_no):
    query = {
        'customer_no': customer_no,
        'installation_no': installation_no
    }
    res_list = list(metadata_collection.find(query).sort('incident_created_at_dt', -1).limit(1))
    if len(res_list) == 0:
        print('No doublet match found')
    else:
        css_object_id = res_list[0]['css_object_id']
        #print(css_object_id)
    # To get incidents as well
    test_incident = metadata_collection.find_one({'css_object_id': css_object_id})
    backtrack_days = 60
    projection = {
        "css_object_id": True,
        "prod_version": True,
        "customer_no": True,
        "customer_name": True,
        "component": True,
        "description": True,
        "incident_created_at_dt": True,
        "solns": True
    }
    query = {
        'prod_version': test_incident['prod_version'],
        'incident_created_at_dt': {'$gt': test_incident['incident_created_at_dt'] - 
                                   timedelta(days=backtrack_days),'$lt': test_incident['incident_created_at_dt']
                                  },
        'customer_no': test_incident['customer_no'],
        'installation_no': test_incident['installation_no']
    }
    test_df = pd.DataFrame(metadata_collection.find(query, projection))
    return(test_df['css_object_id'])
    

In [155]:
get_doublet_incidents(customer_no, installation_no)

0     002075129400003203322022
1     002075129500003089632022
2     002075129500003099102022
3     002075129500003235062022
4     002075129400003324162022
5     002075129400003355412022
6     002075129400003396132022
7     002075129400003396902022
8     002075129500003262802022
9     002075129500003269362022
10    002075129500003370332022
11    002075129400003527732022
12    002075129500003549522022
13    002075129400003708932022
14    002075129400003828372022
15    002075129400003931292022
16    002075129500004290752022
17    002028376700036101062022
Name: css_object_id, dtype: object

# Get commmon solutions for a doublet (ISM API)



In [156]:
def get_doublet_solutions(customer_no, installation_no):
    incident_list = get_doublet_incidents(customer_no, installation_no)
    solution_list = []
    for incident in incident_list:
        sols = get_incident_solutions_2(incident)
        solution_list.append(sols)
    solution_list_flat = [item for sublist in solution_list for item in sublist]
    c = Counter( solution_list_flat ).most_common()
    sols = [word for word, occurrences in c if occurrences > 1]
    return(sols)
    

In [157]:
get_doublet_solutions(customer_no, installation_no)

['1012293', '2989899', '2867786', '2685359']

In [ ]:
%timeit get_doublet_solutions(customer_no, installation_no)

#  Automatic validation scenarios (3)

In [102]:
infodocs_metadata_cloud = mongo_client['IBSO']['infodocs_metadata_cloud_only']
# Scenario 1
release_2202_df = pd.DataFrame(infodocs_metadata_cloud.find({'prod_version': 'SAP S/4HANA CLOUD 2202'}))
duplets_to_remove = release_2202_df[['customer_no', 'installation_no']].drop_duplicates()
all_cloud_df = pd.DataFrame(infodocs_metadata_cloud.find({}))
ind_to_remove = pd.Int64Index([])
for row in duplets_to_remove.iloc:
    ind_to_remove = ind_to_remove.join(all_cloud_df[(all_cloud_df['customer_no'] == row['customer_no']) & (all_cloud_df['installation_no'] == row['installation_no'])].index, how='outer')
ind_to_remove
scenario_1_df = all_cloud_df.drop(index=ind_to_remove)
# Scenario 2
latest_actual_dt = list(infodocs_metadata_cloud.find({'prod_version': 'SAP S/4HANA CLOUD 2202'}).sort('incident_created_at_dt', -1).limit(1))[0]['incident_created_at_dt']
latest_dt = latest_actual_dt - timedelta(days=61)
scenario_2_df = pd.DataFrame(infodocs_metadata_cloud.find({'incident_created_at_dt': {
    '$lt': latest_dt
}}))
# Scenario 3
scenario_3_df = pd.DataFrame(infodocs_metadata_cloud.find({}))


In [104]:
scenario_3_df.head(3)

,_id,component,css_object_id,customer_name,customer_no,description,description_problem_description,incident_created_at_dt,prod_version,Reply,communication_logs,solns,installation_no,system_no
0,635002305b3257a4b003b47d,XX-SER-SAPSMP-LAUNCH,002075129400004638912020,SAP Test Account - aPaul Pharma Tes,0001208936,Test incident for Business User *,Test incident for Business User *. --- Product...,2020-07-15 13:41:18,SAP S/4HANA CLOUD 1506,"Hello, Incidents logged under XX-SER-SAPSMP-LA...",NaN,[2434336],0090316691,800317677
1,635002305b3257a4b003b47e,LOD-MDM-DE,002075129500004950352019,SAP Test Account - aPaul Pharma Tes,0001208936,DO NOT TOUCH - * * (I049611) - SPU function S4...,DO NOT TOUCH - * * (I049611) - SPU function S4...,2019-08-30 08:47:26,SAP S/4HANA CLOUD 1506,sending back to partners DO NOT TOUCH - * * (I...,NaN,[12],0090316691,
2,635002305b3257a4b003b47f,CEC-C4F-CPXF-L2,002075129500003794572019,SAP Test Account - aPaul Pharma Tes,0001208936,test incident,test incident. test,2019-07-05 13:38:39,SAP S/4HANA CLOUD 1506,Checking subscription can be closed test incident,NaN,[1239839],0090316691,
